In [23]:
import numpy as np
import re
import time
from stanfordcorenlp import StanfordCoreNLP
K = 13
biasl = int((K-1)/2)
biasr = K - biasl
NUM_OF_RELATION = 7
PATH = "/Users/denhiroshi/Downloads/dataset/SemEval2007-Task4/train/"
Num_of_sentence_all = 0
Num_of_sentence_T = []
Word_table = set()

In [24]:
T = {
    '1' : [],
    '2' : [],
    '3' : [],
    '4' : [],
    '5' : [],
    '6' : [],
   '7' : [],
}

P = {
    '1' : [],
    '2' : [],
    '3' : [],
    '4' : [],
    '5' : [],
    '6' : [],
    '7' : [],
}

for i in range(1,8):
    with open(PATH + 'relation-%s-train.txt' % (i)) as f:
        Num_of_sentence = 0
        line1 = f.readline()[5:-1]
        while line1 :
            if(len(line1) > 3 and (line1[0:1] >= '0' and line1[0:1] <= '9')):
                line2 = f.readline()
                if line2.find('true') != -1:
                    line2 = line2.split('", ')
                    temp = 0
                    for j in range(len(line2)):
                        if line2[j].find('-') != -1:
                            temp = j
                    start = line2[temp].find('(')
                    T[str(i)].append(line1[5:-2])
                    P[str(i)].append(line2[temp][start+1:line2[temp].find(')')])
                    Num_of_sentence += 1
            line1 = f.readline()
        Num_of_sentence_T.append(Num_of_sentence)

Num_of_sentence_all = sum(Num_of_sentence_T)
Num_of_sentence_all,Num_of_sentence_T

(467, [72, 71, 84, 54, 57, 65, 64])

In [25]:
print(T['1'][0])
P['1'][0]

Earplugs relieve the <e1>discomfort</e1> from <e2>traveling</e2> with a cold allergy or sinus condition.


'e2,e1'

In [33]:
Arrays = []
Tensors = []
avg_length = 0
for i in range(467):
    a = np.loadtxt('arrays'+str(i)+'.csv',dtype=np.float64,delimiter=',')
    b = np.fromfile('tensors'+str(i)+'.dat',dtype=np.float64).reshape((467,a.shape[0],a.shape[1]))
    Arrays.append(a)
    Tensors.append(b.transpose((2,1,0)))
    avg_length += Tensors[i].shape[1]
avg_length /= len(Tensors)
Tensors[0].shape,int(avg_length)

((13, 13, 467), 17)

In [27]:
import torch
from torch.autograd import Function
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tqdm import tqdm
import time
import math
import logging
import pandas as pd
lr = 0.05
n_epoches = 100
tl.set_backend('pytorch')
help(tl.set_backend)

Help on function set_backend in module tensorly:

set_backend(backend_name)
    Sets the backend for TensorLy
    
        The backend will be set as specified and operations will used that backend
    
    Parameters
    ----------
    backend_name : {'mxnet', 'numpy', 'pytorch', 'tensorflow', 'cupy'}, default is 'numpy'



Using pytorch backend.


In [131]:
def get_loss(data,Y):
    torch.set_grad_enabled(True)
    a,b,c = parafac(data,data.shape[0])
    loss_fn = torch.nn.SmoothL1Loss(reduce=False, size_average=False)
    loss_1 = loss_fn(a,Y)
    loss_2 = loss_fn(b,Y)
    loss = (loss_1+loss_2)/2
    return loss.mean()

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)

def timeSince(start, percent):
    now = time.time()
    s = now - start
    r = s / (percent)
    return "%s (- %s)" % (asMinutes(s), asMinutes(r - s))

In [160]:
from torch import nn
class WV_model_1(nn.Module):
    def __init__(self):
        super(WV_model_1, self).__init__()
        torch.set_default_dtype(torch.double)
        self.weight = 
        self.L = nn.Linear(in_features=467,out_features=1,bias=True)
        self.Sigmoid = nn.ReLU()

    def forward(self, sentence):
        torch.set_grad_enabled(True)
        print(f'sentence:{sentence}')
        a = self.L(sentence)
        print(a)
        a_sigmoid = self.Sigmoid(a)
        sentence_T = torch.transpose(sentence,1,0)
        res = torch.mul(sentence_T,a_sigmoid) 
        return res

In [161]:
from random import shuffle
from torch.utils.data import DataLoader
BATCH_SIZE = 50
Embeddings = []
inputs = []
for i in range(467):
    torch.set_grad_enabled(True)
    a = torch.from_numpy(Tensors[i])
    b = torch.from_numpy(Arrays[i])
    inputs.append((a, b))
shuffle(inputs)
inputs_train = inputs[0:int(0.7*len(inputs))]
inputs_test = inputs[int(0.7*len(inputs)):-1]
inputs_test_x = []
inputs_test_y = []
for i in range(len(inputs_test)):
    inputs_test_x.append(inputs_test[i][0])
    inputs_test_y.append(inputs_test[i][1])

start = time.time()
net = WV_model_1()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
for epoch in range(200): 
    loss_sum = 0
    shuffle(inputs_train)
    for (batch_x, batch_y) in inputs:
        pred = net(batch_x.double())
        loss = get_loss(pred,batch_y) # 计算loss
        loss_sum += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"epoch: {epoch}, loss: {loss_sum/len(inputs_train)} ")

sentence:tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0.,

/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorly/backend/numpy_backend.py:246: RuntimeWarning: invalid value encountered in true_divide
  U = np.dot(matrix, V) * 1/S[None, :]


sentence:tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0.,

sentence:tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0.,

AssertionError: 

In [114]:
import csv
headers = ['id', 'Text', 'label']
csv_data = []

csv_data_counter = 0
tag = [72, 71, 84, 54, 57, 65, 64]
Y = np.zeros(467)
print(Y.shape)
Y[0 : tag[0]] = 0
Y[tag[0] : tag[0]+tag[1]] = 1
Y[tag[0]+tag[1] : tag[0]+tag[1]+tag[2]] = 2
Y[tag[0]+tag[1]+tag[2] : tag[0]+tag[1]+tag[2]+tag[3]] = 3
Y[tag[0]+tag[1]+tag[2]+tag[3] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]] = 4
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]] = 5
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]+tag[6]] = 6
Y = Y.tolist()

for i in range(len(Tensors)):
    Tensor = torch.from_numpy(Tensors[i])
    target = Y[i]
    a,b,c = parafac(Tensor+0.05,Tensor.shape[0])
    predict = net(Tensor+0.05)
    csv_data.append((csv_data_counter,torch.mm(c,predict),target))
    csv_data_counter+=1
    print(f'csv_data: {csv_data_counter}')
    
shuffle(csv_data)

csv_data

(467,)


RuntimeError: matrices expected, got 2D, 3D tensors at /Users/administrator/nightlies/pytorch-1.0.0/wheel_build_dirs/wheel_3.7/pytorch/aten/src/TH/generic/THTensorMath.cpp:935

In [ ]:
import torchvision
import matplotlib.pyplot as plt

class RNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.LSTM(
            input_size = 467,       # rnn input
            hidden_size = 128,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.out=torch.nn.Linear(in_features=128,out_features=7)

    def forward(self,x):
        output,(h_n,c_n)=self.rnn(x.float())
        output_in_last_timestep=h_n[-1,:,:]
        x=self.out(output_in_last_timestep)
        return x

In [46]:
# 创建数据加载器
loader = DataLoader(
    dataset = train_inputs, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)

net = RNN()
optimizer = torch.optim.Adam(net.parameters(),lr=0.001)
loss_F = F.cross_entropy
print(net)
for epoch in range(200): 
    for step, (batch_id,batch_x, batch_y) in enumerate(loader):
        pred = net(batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step % 50 == 0: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(test_inputs_x)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == test_inputs_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

AttributeError: module 'numpy' has no attribute 'read_csv'